In [2]:
!pip install git+https://github.com/lalitpagaria/haystack.git@implement_RAG
!pip install urllib3==1.25.4

  Cloning https://github.com/lalitpagaria/haystack.git (to revision implement_RAG) to /tmp/pip-req-build-j7v_rax4
  Running command git clone -q https://github.com/lalitpagaria/haystack.git /tmp/pip-req-build-j7v_rax4
  Running command git checkout -b implement_RAG --track origin/implement_RAG
  Switched to a new branch 'implement_RAG'
  Branch 'implement_RAG' set up to track remote branch 'implement_RAG' from 'origin'.
  Created wheel for farm-haystack: filename=farm_haystack-0.4.0-cp36-none-any.whl size=91365 sha256=1ea2b4592f83a2978cef7f83e5a5cf989387b2abd96cd6a264eaf16121af7d66
  Stored in directory: /tmp/pip-ephem-wheel-cache-pqjt7vxs/wheels/ed/03/62/5e4631caace5a1832a21af6a63b621e52159737b4c0cab9ecc
Successfully built farm-haystack
     |████████████████████████████████| 133kB 8.6MB/s 
ERROR: kaggle 1.5.8 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.

In [5]:
from haystack import Document
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.generator.transformers import RAGenerator
from haystack.retriever.dense import DensePassageRetriever

10/22/2020 10:55:08 - INFO - faiss -   Loading faiss with AVX2 support.
10/22/2020 10:55:08 - INFO - faiss -   Loading faiss.


In [6]:
documents = [
    Document(
        text="""Berlin is Germany capital"""
    ),
    Document(
        text="""Berlin is the capital and largest city of Germany by both area and population.""",
    )
]

In [7]:
document_store = FAISSDocumentStore()
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  use_gpu=False, embed_title=True,
                                  remove_sep_tok_from_untitled_passages=True)
generator = RAGenerator(retriever=retriever)

10/22/2020 10:55:13 - INFO - filelock -   Lock 139691029815192 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


10/22/2020 10:55:14 - INFO - filelock -   Lock 139691029815192 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


10/22/2020 10:55:14 - INFO - filelock -   Lock 139691029957768 acquired on /root/.cache/torch/transformers/4b05580c0bfb2b640a50c1c6ae3fe9bca923871a29e0182927c086905d6c4c47.7652e92693c670fb8dfd7ec1f9191e3f82673742ff6a86cde9133a4ea6002ced.lock


10/22/2020 10:55:15 - INFO - filelock -   Lock 139691029957768 released on /root/.cache/torch/transformers/4b05580c0bfb2b640a50c1c6ae3fe9bca923871a29e0182927c086905d6c4c47.7652e92693c670fb8dfd7ec1f9191e3f82673742ff6a86cde9133a4ea6002ced.lock
10/22/2020 10:55:15 - INFO - filelock -   Lock 139691029579424 acquired on /root/.cache/torch/transformers/8fdd0d2838c23f921379f2b0322aecf406cbdaa97ffecc544e3a1d49a7c302bd.6f90756c59007364d7842118056ad653f39f4d340fbe20bcc04037d2a45cb0f7.lock


10/22/2020 10:55:40 - INFO - filelock -   Lock 139691029579424 released on /root/.cache/torch/transformers/8fdd0d2838c23f921379f2b0322aecf406cbdaa97ffecc544e3a1d49a7c302bd.6f90756c59007364d7842118056ad653f39f4d340fbe20bcc04037d2a45cb0f7.lock


...

10/22/2020 10:56:29 - INFO - filelock -   Lock 139691009024744 released on /root/.cache/torch/transformers/ecaef97670694deb80161d2734505878a7b38565c89fd3e1c6441899f4e6a51e.6e217123a3ada61145de1f20b1443a1ec9aac93492a4bd1ce6a695935f0fd97a.lock


10/22/2020 10:56:29 - INFO - filelock -   Lock 139691029814408 acquired on /root/.cache/torch/transformers/189b603398b8d6afb7d5377372dc4cf0bb48742f67be7bedbe775ee86b356f52.d596a549211eb890d3bb341f3a03307b199bc2d5ed81b3451618cbcb04d1f1bc.lock


10/22/2020 10:56:30 - INFO - filelock -   Lock 139691029814408 released on /root/.cache/torch/transformers/189b603398b8d6afb7d5377372dc4cf0bb48742f67be7bedbe775ee86b356f52.d596a549211eb890d3bb341f3a03307b199bc2d5ed81b3451618cbcb04d1f1bc.lock


10/22/2020 10:56:31 - INFO - filelock -   Lock 139691029578024 acquired on /root/.cache/torch/transformers/06fe449ffe41cbe16aeb1f5976989313464a3c44a605e9a8b91bf6440dfa6026.696574d8c17eafbac08f43f01e951252057f8feb133b64a33b76d4c47d65367a.lock


10/22/2020 10:57:42 - INFO - filelock -   Lock 139691029578024 released on /root/.cache/torch/transformers/06fe449ffe41cbe16aeb1f5976989313464a3c44a605e9a8b91bf6440dfa6026.696574d8c17eafbac08f43f01e951252057f8feb133b64a33b76d4c47d65367a.lock


In [8]:
document_store.delete_all_documents()
document_store.write_documents(documents)
document_store.update_embeddings(retriever=retriever)

10/22/2020 10:57:59 - INFO - haystack.document_store.faiss -   Updating embeddings for 2 docs...
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/haystack/retriever/dense.py:199: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  no_title_indices = torch.nonzero(1

In [9]:
question = "What is capital of the Germany?"
retriever_results = retriever.retrieve(query=question, top_k=2)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [10]:
predicted_result = generator.predict(question=question, documents=retriever_results, top_k=1)

print("By Haystack=", predicted_result["answers"][0]["answer"])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


By Haystack=  berlin
